In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error

In [2]:
import my_models as my_models

In [3]:
net3_ped1 = my_models.Net3_ped1()
net3_ped2 = my_models.Net3_ped2()
net3_ped3 = my_models.Net3_ped3()
net3_ped4 = my_models.Net3_ped4()
net3_ped5 = my_models.Net3_ped5()
net3_ped6 = my_models.Net3_ped6()
net3_ped7 = my_models.Net3_ped7()
net3_ped8 = my_models.Net3_ped8()
net3_ped9 = my_models.Net3_ped9()
net3_ped_repulse_total = my_models.Net3_ped_repulse_total(net3_ped1, net3_ped2, net3_ped3, net3_ped4, net3_ped5,
                                                net3_ped6, net3_ped7, net3_ped8, net3_ped9)
net3_attract = my_models.Net3_attract()
net3_repulse = my_models.Net3_repulse()
net3_ped_group = my_models.Net3_ped_group()

In [7]:
path_ped_repulse = "weights/Net3_ped_repulse_total.pth"
path_attract = "weights/net3_attract.pth"
path_repulse = "weights/net3_repulse.pth"
path_group = "weights/net3_ped_group.pth"

In [8]:
net3_ped_repulse_total.load_state_dict(torch.load(path_ped_repulse))
net3_attract.load_state_dict(torch.load(path_attract))
net3_repulse.load_state_dict(torch.load(path_repulse))
net3_ped_group.load_state_dict(torch.load(path_group))

<All keys matched successfully>

In [9]:
net3_attract

Net3_attract(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (fc3): Linear(in_features=20, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=2, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)

In [10]:
net3_a_rb = my_models.Net3_a_rb(net3_attract, net3_repulse)
# net3_a_rb

In [11]:
path_net3_a_rb = "weights/net3_a_rb.pth"
net3_a_rb.load_state_dict(torch.load(path_net3_a_rb))

<All keys matched successfully>

In [12]:
net3_a_rb_rp = my_models.Net3_a_rb_rp(net3_attract, net3_repulse, net3_ped_repulse_total)

In [14]:
path_net3_a_rb_rp = "weights/net3_a_rb_rp.pth"
net3_a_rb_rp.load_state_dict(torch.load(path_net3_a_rb_rp))

<All keys matched successfully>

In [15]:
net3_complete = my_models.Net3_complete(net3_attract, net3_repulse, net3_ped_repulse_total, net3_ped_group)
# print(net3_complete)

In [16]:
path_net3_complete = "weights/net3_complete2.pth"

In [17]:
net3_complete.load_state_dict(torch.load(path_net3_complete))

<All keys matched successfully>

In [18]:
from scipy.interpolate import interp1d

def input_interpolated(temp_xy):
    # takes the 3 timesteps of inputs and rescales it into 10 timesteps
    # returns as (10, 2)
    temp_x = np.array((temp_xy[0], temp_xy[2], temp_xy[4]))
    temp_y = np.array((temp_xy[1], temp_xy[3], temp_xy[5]))
    x_total = []
    y_total = []
    for p in range(2):
        x = temp_x[0+p:2+p]
        y = temp_y[0+p:2+p]
        if (x[0] == x[-1] and y[0] == y[-1]):
            # if both x values are same, we do this to avoid nan given by interp1d
            x_new = np.linspace(x[0], x[-1], num=5)
            y_new = np.linspace(y[0], y[-1], num=5)
        else:            
            f2 = interp1d(x, y)
            x_new = np.linspace(x[0], x[-1], num=5)
            
            y_new = f2(x_new)
        x_total.append(x_new)
        y_total.append(y_new)
    x_total = np.array(x_total)
    x_total = x_total.flatten()
    y_total = np.array(y_total)
    y_total = y_total.flatten()
    xy_input = []
    for p in range(len(x_total)):
        temp_array = [x_total[p], y_total[p]]
        xy_input.append(temp_array)
    xy_input = np.array(xy_input)
    return xy_input

In [19]:
# test_df_path = "C:/Users/sakif/jupyter projects/thesis_test/modular test/Untitled Folder/eth_data_prepared.csv"
# eth_data_prepared_normal.csv
# fps=10
# delta_t = 0.1

In [20]:
test_df_path = "data/test_data/eth_data_prepared_normal3.csv"
# delta_t = 0.4

In [21]:
test_df = pd.read_csv(test_df_path, index_col=None)

In [22]:
test_df.head()

,frame_id,ped_no,pos_x,pos_y,vel_x,vel_y,rotation_angle,eta_group_x,eta_group_y,bound_x,...,n_w_y21,n_w_y22,n_w_y23,n_w_y24,n_w_y25,n_w_y26,n26_y,d_w_bound,n_w_bound_x,n_w_bound_y
0,780,1,8.456844,3.588066,1.671714,0.176292,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.906774,0.949485,0.313814
1,786,1,9.125530,3.658583,1.662877,0.326723,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.564877,0.954067,0.299594
2,792,1,9.787146,3.849445,1.683334,0.371084,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.253295,0.954537,0.298094
3,798,1,10.472197,3.955450,1.598568,0.264795,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.939286,0.957302,0.289091
4,804,1,11.066000,4.061280,1.574526,0.456390,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.538544,0.959047,0.283249


In [23]:
np.min(test_df["eta_group_x"])

-0.9999998555153242

In [24]:
from collections import Counter

In [25]:
tempppp = Counter(np.array(test_df['ped_no']))
# tempppp
# 171

In [26]:
len(np.unique(np.array(test_df['ped_no'])))
# 51: 64

360

In [27]:
# dataframe for individual pedestrian 
test_df_indiv = test_df.loc[test_df['ped_no'] == 2]
# test_df_indiv = test_df_indiv.loc[test_df_indiv['frame_id'] >= 846]

In [28]:
input_xy_temp = np.array(test_df_indiv[["pos_x", "pos_y"]])

In [29]:
# input_xy_temp

In [30]:
test_df_indiv.head()

,frame_id,ped_no,pos_x,pos_y,vel_x,vel_y,rotation_angle,eta_group_x,eta_group_y,bound_x,...,n_w_y21,n_w_y22,n_w_y23,n_w_y24,n_w_y25,n_w_y26,n26_y,d_w_bound,n_w_bound_x,n_w_bound_y
5,804,2,13.017548,5.782591,-2.324444,-0.076606,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.941039,0.933757,0.357907
7,810,2,12.087770,5.751949,-1.589052,-0.065971,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.065427,0.925172,0.379547
9,816,2,11.746306,5.729815,-1.140766,0.105373,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.741579,0.921888,0.387457
10,822,2,11.175158,5.836247,-1.640021,0.179235,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.260445,0.911481,0.411343
11,828,2,10.434290,5.873203,-1.406129,0.419050,0.0,0.0,0.0,0.793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.605294,0.899097,0.437749


In [31]:
from matplotlib import pyplot as plt

In [33]:
def speed_finder(positions):
    positions = positions.view((10, 2))
    temp_array = []
    for k in reversed(range(10)):
        # print(k)
        temp_positions = positions[k] - positions[k - 1]
        temp_positions = torch.norm(temp_positions)
        temp_array.insert(0, temp_positions)
    return torch.Tensor([temp_array])

In [34]:
import math

In [35]:
# make a class prediction for one row of data
#  e, x_attract1, x_attract2, x_bound, x1, x2, x3, x4, x5, x6, x7, x8, x9, x_group, x_speed
def predict_synthetic(model, delta_t, last_velocity, temp_input, temp_goal, temp_n1, temp_n2, temp_n3, temp_n4, temp_n5,
                      temp_n6, temp_n7, temp_n8, temp_n9, temp_n_bound, temp_n_group, temp_n_centroid, indicator):

    previous_velocity = last_velocity
    previous_speed = np.linalg.norm(previous_velocity)
    temp_input2 = temp_input - temp_input[:, 0]
    temp_input_1 = speed_finder(temp_input2)/delta_t
    x_speed = temp_input_1[:, -1:]
    # make prediction
    # yhat = net3_complete(goal2, x_attract1, x_attract2, x_bound, x1, x2, x3, x4, x5, x6, x7, x8, x9, x_group, x_speed)
    if indicator == 0: 
        # for net3_complete
        yhat = model(temp_goal, temp_input_1, temp_input2, temp_n_bound, temp_n1, temp_n2, temp_n3, temp_n4, temp_n5, 
                     temp_n6, temp_n7, temp_n8, temp_n9, temp_n_group, temp_n_centroid)
    elif indicator == 1:
        # for model with attract and repulse from obstacle
        yhat = model(temp_goal, temp_input_1, temp_input2, temp_n_bound)
    elif indicator == 2:
        # for model with attract, repulse from obstacle and other peds
        yhat = model(temp_goal, temp_input_1, temp_input2, temp_n_bound, temp_n1, temp_n2, temp_n3, temp_n4, temp_n5, 
                     temp_n6, temp_n7, temp_n8, temp_n9)
    elif indicator == 3:
        yhat = model(temp_input_1, temp_input2, temp_goal)
        
    
    # retrieve numpy array
    yhat = yhat.detach().numpy()

    desired_velocity = previous_velocity + (yhat * delta_t)
    desired_speed = np.linalg.norm(desired_velocity)
    
    # this section is to cap the velocity to a certain value
    factor = np.minimum(1.0, (1.3 * previous_speed) / desired_speed)
    if desired_speed == 0:
        factor = 0.0
    # print(factor)
    current_velocity = desired_velocity * factor
    next_position = temp_input[0, -2:].detach().numpy() + (current_velocity * delta_t)
     
    return yhat, next_position

In [38]:
def nearest_neighbour_finder(temp_neighbours):
    a = temp_neighbours[0]
    b = np.argsort(a[np.nonzero(a)])[:9]
    b = b + 1
    label_dw = "d_w"
    label_nw_x = "n_w_x"
    label_nw_y = "n_w_y"
    label_dw_list = []
    label_nw_x_list = []
    label_nw_y_list = []
    for r in range(len(b)):
        label_dw2 = label_dw + str(b[r])
        label_nw_x2 = label_nw_x + str(b[r])
        label_nw_y2 = label_nw_y + str(b[r])
        label_dw_list.append(label_dw2)
        label_nw_x_list.append(label_nw_x2)
        label_nw_y_list.append(label_nw_y2)

    label_dw_list = np.array(label_dw_list)
    label_nw_x_list = np.array(label_nw_x_list)
    label_nw_y_list = np.array(label_nw_y_list)
    return label_dw_list, label_nw_x_list, label_nw_y_list

In [40]:
def goal_estimator2(given_dest, input_xy):
    goal_estimated = (given_dest - input_xy[0])/np.linalg.norm((given_dest - input_xy[0]))
    goal_estimated = torch.Tensor([goal_estimated])
    return goal_estimated

In [41]:
def get_last_disp(y_true, y_prediction):
    last_disp = np.linalg.norm((y_true[-1, :] - y_prediction[-1, :]))
    return last_disp

def get_eucledian(true_values, predicted_values):
    eucledian_v = np.linalg.norm((true_values - predicted_values), axis =1)
    eucledian_v = np.mean(eucledian_v)
    return eucledian_v

In [42]:
dest_df = pd.read_csv("filter/eth_estimated_dests2.csv", index_col=0)

In [43]:
dest_df.head()

,dest_x,dest_y
1,-6.555308,11.867515
2,-2.000000,5.856603
3,-2.000000,5.856603
4,15.107170,5.565930
5,15.107170,5.565930


In [44]:
estimated_dest_list = np.array(dest_df[["dest_x", "dest_y"]])

In [45]:
estimated_dest_list.shape

(360, 2)

In [95]:
# to predict all the trajectories, 0.1 seconds of prediction at a time
# input of one seconds
# next step: make it into a function
# function to get predictions of trajectories, recursively
def get_predictions(step_size, delta_t, temp_cycle, input_xy, velocity, n1, n2, n3, n4, n5, n6, n7, n8, n9, n_bound,
                    n_group, n_centroid, estimated_dest, indicator):
    # dest_list = np.array(((-2, 5.8566027), (-6.5902743, 0.065724367), (-6.5553084, 11.867515), (15.10717, 5.5659299)))
    step_size = step_size
    delta_t = delta_t
    temp_cycle = temp_cycle
    predicted_trajs = []
    real_trajs = []
    force_pred = []
    indicator2 = indicator
    interpolate = True
    my_goal = goal_estimator2(estimated_dest, input_xy)
    # len(input_xy)
    end_cycle = len(input_xy)
    for p in range(5, end_cycle, step_size):
        # print("timesteps: ", p)
        start_cycle = temp_cycle + p
        # print(start_cycle + step_size)
        if (start_cycle + step_size) > end_cycle: 
            # print("i am here")
            continue

        if (temp_cycle != 10):
            if not interpolate:
                temp_input5 = np.concatenate((np.zeros(14), input_xy[(start_cycle - temp_cycle):start_cycle].flatten()))
                temp_input = torch.Tensor([temp_input5])
            else:
                temp_input_total = input_xy[(start_cycle - temp_cycle):start_cycle].flatten()
                # print(start_cycle - temp_cycle, start_cycle)
                temp_input5 = input_interpolated(temp_input_total).flatten()
                temp_input = torch.Tensor([temp_input5])
        else: 
            temp_input = torch.Tensor([input_xy[(start_cycle - temp_cycle):start_cycle].flatten()])

        temp_real = input_xy[(start_cycle):(start_cycle + step_size)]
        real_trajs.append(temp_real)
        prediction_list = []
        next_position_list = []
        # print("input: ", temp_input, "\n")
        for x in range(step_size):
            if x == 0:
                temp_input3 = temp_input
            else: 
                if not interpolate:
                    temp_input3 = torch.cat((temp_input[:, 2:], torch.Tensor(next_position)), axis=1)
                    temp_input = temp_input3
                else:
                    temp_input_total = np.concatenate((temp_input_total[2:], next_position.flatten()))
                    temp_input3 = input_interpolated(temp_input_total).flatten()
                    temp_input3 = torch.Tensor([temp_input3])
                    temp_input = temp_input3

            temp_velocity = velocity[(start_cycle -1 + x)]
            temp_goal = my_goal
            temp_n1 = torch.Tensor([n1[start_cycle + x].flatten()])
            temp_n2 = torch.Tensor([n2[start_cycle + x].flatten()])
            temp_n3 = torch.Tensor([n3[start_cycle + x].flatten()])
            temp_n4 = torch.Tensor([n4[start_cycle + x].flatten()])
            temp_n5 = torch.Tensor([n5[start_cycle + x].flatten()])
            temp_n6 = torch.Tensor([n6[start_cycle + x].flatten()])
            temp_n7 = torch.Tensor([n7[start_cycle + x].flatten()])
            temp_n8 = torch.Tensor([n8[start_cycle + x].flatten()])
            temp_n9 = torch.Tensor([n9[start_cycle + x].flatten()])
            temp_n_bound = torch.Tensor([n_bound[start_cycle + x].flatten()])
            temp_n_group = torch.Tensor([n_group[start_cycle + x].flatten()])
            temp_n_centroid = torch.Tensor([n_centroid[start_cycle + x].flatten()])
            
            temp_model = net3_complete
            
            if indicator2 == 0:
                prediction1, next_position = predict_synthetic(net3_complete, delta_t, temp_velocity,  temp_input3, temp_goal, 
                                                           temp_n1, temp_n2, temp_n3, temp_n4, temp_n5, temp_n6, temp_n7, 
                                                           temp_n8, temp_n9, temp_n_bound, temp_n_group, temp_n_centroid,
                                                           indicator2)
            elif indicator2 == 1:
                prediction1, next_position = predict_synthetic(net3_a_rb, delta_t, temp_velocity,  temp_input3, temp_goal, 
                                                           temp_n1, temp_n2, temp_n3, temp_n4, temp_n5, temp_n6, temp_n7, 
                                                           temp_n8, temp_n9, temp_n_bound, temp_n_group, temp_n_centroid,
                                                           indicator2)
            elif indicator2 == 2:
                prediction1, next_position = predict_synthetic(net3_a_rb_rp, delta_t, temp_velocity,  temp_input3, temp_goal, 
                                                           temp_n1, temp_n2, temp_n3, temp_n4, temp_n5, temp_n6, temp_n7, 
                                                           temp_n8, temp_n9, temp_n_bound, temp_n_group, temp_n_centroid,
                                                           indicator2)
                
            elif indicator2 == 3: 
                prediction1, next_position = predict_synthetic(net3_attract, delta_t, temp_velocity,  temp_input3, temp_goal, 
                                                           temp_n1, temp_n2, temp_n3, temp_n4, temp_n5, temp_n6, temp_n7, 
                                                           temp_n8, temp_n9, temp_n_bound, temp_n_group, temp_n_centroid,
                                                           indicator2)
            else:
                pass

            prediction_list.append(prediction1)
            next_position_list.append(next_position)


        prediction_list = np.array(prediction_list)
        next_position_list = np.array(next_position_list)
        predicted_trajs.append(next_position_list)
        force_pred.append(prediction_list)

    predicted_trajs = np.array(predicted_trajs)
    force_pred = np.array(force_pred)
    real_trajs = np.array(real_trajs)

    predicted_trajs_reshaped = np.reshape(predicted_trajs, (int((len(predicted_trajs.flatten())/2)), 2))
    real_trajs_reshaped = np.reshape(real_trajs, (int((len(real_trajs.flatten())/2)), 2))

    eucledian_temp_list = []
    for k in range(0, len(predicted_trajs_reshaped), step_size):
        eucledian_temp = get_eucledian(real_trajs_reshaped[0+k:12+k], predicted_trajs_reshaped[0+k:12+k])
        eucledian_temp_list.append(eucledian_temp)
    eucledian_temp_list = np.array(eucledian_temp_list)
    final_eucledian_dist = np.mean(eucledian_temp_list)
    final_disp = get_last_disp(real_trajs_reshaped, predicted_trajs_reshaped)

    return final_eucledian_dist, final_disp, real_trajs_reshaped, predicted_trajs_reshaped

In [105]:
ped_array = np.unique(np.array(test_df['ped_no']))
eucledian_for_all = []
displacement_for_all = []
temp_count = 0
temp_count2 = 0
real_traj_list = np.zeros((1, 2))
pred_traj_list = np.zeros((1, 2))
time_steps_list = np.zeros((1,))
print(len(ped_array))
for j in range(len(ped_array)):
    # if j == 5: break
    test_df_indiv = test_df.loc[test_df['ped_no'] == ped_array[j]]
    
    input_xy = np.array(test_df_indiv[["pos_x", "pos_y"]]).astype('float32')
    # print("pedestrian no: ", ped_array[j], "\n")
    if len(input_xy) < 20: 
        # print("length of input_xy", len(input_xy), "\n")
        # print("skipped")
        temp_count2 += 1
        continue

    temp_neighbours = np.array(test_df_indiv[["d_w1", "d_w2", "d_w3", "d_w4", "d_w5", "d_w6", "d_w7", "d_w8", 
                                              "d_w9", "d_w10", "d_w11", "d_w12", "d_w13", "d_w14", "d_w15", 
                                              "d_w16", "d_w17", "d_w18", "d_w19", "d_w20", "d_w21", "d_w22", 
                                              "d_w23", "d_w24", "d_w25", "d_w26"]]).astype('float32')
    neighbour_test = temp_neighbours[0]

    if len(neighbour_test[np.nonzero(neighbour_test)]) > 9:
        temp_count += 1
        # print("we are here")
        dw_list, nw_x_list, nw_y_list = nearest_neighbour_finder(temp_neighbours)
        # print(dw_list)

        n1 = np.array(test_df_indiv[[dw_list[0], nw_x_list[0], nw_y_list[0]]]).astype('float32')
        n2 = np.array(test_df_indiv[[dw_list[1], nw_x_list[1], nw_y_list[1]]]).astype('float32')
        n3 = np.array(test_df_indiv[[dw_list[2], nw_x_list[2], nw_y_list[2]]]).astype('float32')
        n4 = np.array(test_df_indiv[[dw_list[3], nw_x_list[3], nw_y_list[3]]]).astype('float32')
        n5 = np.array(test_df_indiv[[dw_list[4], nw_x_list[4], nw_y_list[4]]]).astype('float32')
        n6 = np.array(test_df_indiv[[dw_list[5], nw_x_list[5], nw_y_list[5]]]).astype('float32')
        n7 = np.array(test_df_indiv[[dw_list[6], nw_x_list[6], nw_y_list[6]]]).astype('float32')
        n8 = np.array(test_df_indiv[[dw_list[7], nw_x_list[7], nw_y_list[7]]]).astype('float32')
        n9 = np.array(test_df_indiv[[dw_list[8], nw_x_list[8], nw_y_list[8]]]).astype('float32')
    else:
        n1 = np.array(test_df_indiv[["d_w1", "n_w_x1", "n_w_y1"]]).astype('float32')
        n2 = np.array(test_df_indiv[["d_w2", "n_w_x2", "n_w_y2"]]).astype('float32')
        n3 = np.array(test_df_indiv[["d_w3", "n_w_x3", "n_w_y3"]]).astype('float32')
        n4 = np.array(test_df_indiv[["d_w4", "n_w_x4", "n_w_y4"]]).astype('float32')
        n5 = np.array(test_df_indiv[["d_w5", "n_w_x5", "n_w_y5"]]).astype('float32')
        n6 = np.array(test_df_indiv[["d_w6", "n_w_x6", "n_w_y6"]]).astype('float32')
        n7 = np.array(test_df_indiv[["d_w7", "n_w_x7", "n_w_y7"]]).astype('float32')
        n8 = np.array(test_df_indiv[["d_w8", "n_w_x8", "n_w_y8"]]).astype('float32')
        n9 = np.array(test_df_indiv[["d_w9", "n_w_x9", "n_w_y9"]]).astype('float32')

    input_xy = np.array(test_df_indiv[["pos_x", "pos_y"]]).astype('float32')

    n_bound = np.array(test_df_indiv[["d_w_bound", "n_w_bound_x", "n_w_bound_y"]]).astype('float32')
    # n_group = np.array(test_df_indiv[["centroid_dist", "rotation_angle"]]).astype('float32')
    n_group = np.array(test_df_indiv["rotation_angle"]).astype('float32')
    n_centroid = np.array(test_df_indiv[["eta_group_x", "eta_group_y"]]).astype('float32')

    velocity = np.array(test_df_indiv[["vel_x", "vel_y"]]).astype('float32')

    step_size = 12
    delta_t = 0.4
    temp_cycle = 3
    estimated_destination = estimated_dest_list[j]
    indicator3 = 0
    # 0 = net3_complete, 1 = attract + obstacle, 2 = attract + obst + ped
    # print(estimated_destination)
    # temp_clcle = the cycle from where the prediction actually stats
    # input_xy, velocity, n1, n2, n3, n4, n5, n6, n7, n8, n9, n_bound, n_group

    eucleid, disp, real_traj_new, pred_traj_new = get_predictions(step_size, delta_t, temp_cycle, input_xy, velocity, n1, n2,
                                                                  n3, n4, n5, n6, n7, n8, n9, n_bound, n_group, n_centroid, 
                                                                  estimated_dest=estimated_destination, indicator=indicator3)
    # print(eucleid, disp)
    temp_timestep = np.array(test_df_indiv["frame_id"][:len(real_traj_new)])
    # print(test_df_indiv["frame_id"][:len(real_traj_new)].to_list())
    # print(temp_timestep.shape)
    real_traj_list = np.concatenate((real_traj_list, real_traj_new), axis=0)
    pred_traj_list = np.concatenate((pred_traj_list, pred_traj_new), axis=0)
    # real_traj_list.append(real_traj_new.flatten())
    time_steps_list = np.concatenate((time_steps_list, temp_timestep), axis=0)
    
    if not math.isnan(eucleid):
        eucledian_for_all.append(eucleid)
        displacement_for_all.append(disp)

eucledian_for_all = np.array(eucledian_for_all)
displacement_for_all = np.array(displacement_for_all)
# real_traj_list = np.array(real_traj_list)

# print()
print(temp_count2)
print("mean eucledian distance: ", np.mean(eucledian_for_all), "\n")
print("mean final displacement: ", np.mean(displacement_for_all), "\n")
real_traj_list = real_traj_list[1:]
pred_traj_list = pred_traj_list[1:]
time_steps_list = time_steps_list[1:]
print(len(real_traj_list))
print(len(time_steps_list))
print(len(pred_traj_list))
        # print("pedestrian no: ", ped_array[j], "\n")
    # if not math.isnan()
        

360
89
mean eucledian distance:  0.08946255 

mean final displacement:  0.1326341 

4224
4224
4224


In [97]:
my_dict = {'frame_id': time_steps_list, 'real_x': real_traj_list[:, 0], 'real_y': real_traj_list[:, 1], 
           'pred_x': pred_traj_list[:, 0], 'pred_y': pred_traj_list[:, 1]}

In [98]:
my_df = pd.DataFrame(data=my_dict, index=None)

In [99]:
my_df.head()

,frame_id,real_x,real_y,pred_x,pred_y
0,804.0,8.552751,6.374027,8.581429,6.336385
1,810.0,8.097887,6.481037,8.094962,6.449198
2,816.0,7.635367,6.548341,7.642897,6.540567
3,822.0,7.167091,6.616482,7.184125,6.612502
4,828.0,6.734173,6.641461,6.740154,6.663274


In [100]:
frame_id_array = np.unique(my_df['frame_id'])
cc_total = 0
interaction_total = 0
for k in range(len(frame_id_array)):
    test_df_temp = my_df.loc[my_df['frame_id'] == frame_id_array[k]]
    temp_pos_list = np.array(test_df_temp[['pred_x', 'pred_y']])
    if len(temp_pos_list) > 1:
        temp_ec_list = np.zeros((1,))
        for j in range(len(temp_pos_list) - 1):
            temp_diff = np.diff(temp_pos_list[j:], axis=0)
            temp_ec = np.linalg.norm(temp_diff, axis=1)
            temp_ec_list = np.concatenate((temp_ec_list, temp_ec), axis=0)
        cc_temp = np.count_nonzero(temp_ec_list[1:] <= 0.1)
        cc_total += cc_temp
        interaction_temp = len(temp_ec_list[1:])
        interaction_total += interaction_temp
print("total collision: ", cc_total)
print("total interaction: ", interaction_total)
print((cc_total/interaction_total)*100)

total collision:  0
total interaction:  10588
0.0


In [ ]:
# approach 1: just get the number of collisions per frame, add them up, then divide by total no of interactions.

In [ ]:
# mean eucledian distance:  0.61246943 

# mean final displacement:  1.2114174 


In [496]:
# for one cycle only
eucledian_distance = get_eucledian(real_trajs_reshaped[:12], predicted_trajs_reshaped[:12])
last_displacement = get_last_disp(real_trajs_reshaped[:12], predicted_trajs_reshaped[:12])
print("eucledian distance: ", eucledian_distance)
print("last displacement: ", last_displacement)

eucledian distance:  0.98691934
last displacement:  1.2742083
